In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook

In [10]:
data = pd.read_csv('dataset.csv')

In [11]:
data = data.sample(frac=1).reset_index(drop=True)

In [ ]:
columns_titles = ["Tweet","Label"]
data = data.reindex(columns=columns_titles)

In [12]:
train_data = data.head(int(225*0.8))

In [13]:
test_data = data.tail(int(225*0.2))

In [14]:
display(train_data)
display(test_data)

,Label,Tweet
0,0,RT @THR: As cases of the coronavirus continue ...
1,0,"RT @AFP: Hong Kong medical workers strike, dem..."
2,0,Recovery rate. China: 10.94% Recovery rate. Ou...
3,0,@bbcchinese https://t.co/ags5OZtPDw
4,0,RT @DHSCgovuk: Four further patients in Englan...
...,...,...
175,1,RT @QuickTake: @business @immigrationPH @Maris...
176,0,"""RT @QuickTake: """"If the government does not s..."
177,0,RT @alfonslopeztena: Hundreds of Hong Kong med...
178,0,@EggTarTisYummy @mertinsantos @kevinleversee @...


,Label,Tweet
181,0,RT @alvertoas: China confirma que el coronavir...
182,1,RT @alfonslopeztena: Hundreds of Hong Kong med...
183,0,RT @HawleyMO: #China thinks efforts to stop #c...
184,0,RT @balajis: The #coronavirus has now killed m...
185,0,"RT @MailOnline: 2,400 medical workers go on st..."
186,0,"RT @cnnphilippines: @CNN Authorities in Wuhan,..."
187,0,RT @BNODesk: Updated charts for coronavirus in...
188,1,"RT @AFP: Hong Kong medical workers strike, dem..."
189,0,@BBCBreaking The risk to the public has not ch...
190,1,RT @annafifield: A person has died of coronavi...


In [15]:
from simpletransformers.classification import ClassificationModel
import logging

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [16]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [18]:
model = ClassificationModel('roberta', 'roberta-base',use_cuda=False)

INFO:filelock:Lock 140198205012240 acquired on /Users/pranavnt/.cache/torch/transformers/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e.lock
Downloading: 100%|██████████| 501M/501M [00:24<00:00, 20.8MB/s]
INFO:filelock:Lock 140198205012240 released on /Users/pranavnt/.cache/torch/transformers/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e.lock
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificati

In [20]:
model.train_model(train_data)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
  0%|          | 0/180 [00:00<?, ?it/s]


TypeError: can't pickle AddedToken objects